In [1]:
import boto3
import json
from boto3.dynamodb.conditions import Key, Attr
import copy
import argparse
import csv
import pandas as pd
import datetime
import sys
import time
from datetime import datetime
import urllib3
import os

s3 = boto3.client('s3')
s3res = boto3.resource('s3')
tr = boto3.client('transcribe')
nlp = boto3.client('comprehend')


### FUNCIONES

In [50]:
def obten_lista_audios(bucket):
    archivos = s3.list_objects(Bucket=bucket) 
    lista_audios = [x["Key"] for x in archivos["Contents"]]

    # startAfter = 'firstlevelFolder/secondLevelFolder'
    # theobjects = s3client.list_objects_v2(Bucket=bucket, StartAfter=startAfter )
    # for object in theobjects['Contents']:      
    #     print object['Key']    

    return lista_audios

def obten_url_transcript(jobName):
    print(jobName)
    while True:
        status = tr.get_transcription_job(TranscriptionJobName=jobName)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        print("Creando transcript de audio")
        time.sleep(15)
    return status['TranscriptionJob']['Transcript']["TranscriptFileUri"]

def consulta_transcript(liga):
    http = urllib3.PoolManager() 
    r = http.request("GET",liga)
    ## Se retorna el texto en UTF-8
    #Se decodifica en UTF-8
    transcript = r.data
    texto = json.loads(r.data)["results"]["transcripts"][0]["transcript"]
    # Se obtiene directo en Bytes el json completo y el texto no por palabras
    return transcript,texto.encode("UTF-8") 


def almacenaS3(ft,tt,file,parametros):
    #Grabar solo texto
    parametros["Key"] = 'Transcripts'+"/"+file+".txt"
    parametros["Body"] = tt
    
    ## Se pasa los argumentos directo del diccionario
    res_textfile =  s3.put_object(**parametros)
    res_http = res_textfile["ResponseMetadata"]["HTTPStatusCode"]
        
    if res_http !=200:
        print("Error HTTP" +str(res_http) +" Posible problema de comunicación al procesar el full json ")
    time.sleep(2)
    
    ## Grabar el json completo en archivo
    parametros["Key"] = 'Transcripts'+"/"+file+"_full.txt"
    parametros["Body"] = ft
    res_textfile = s3.put_object(**parametros)
    res_http = res_textfile["ResponseMetadata"]["HTTPStatusCode"]
    
    if res_http !=200:
        print("Error HTTP" +str(res_http) +" Posible problema de comunicación al procesar el texto plano ")
    time.sleep(2)
    
    return 0

def almacenaS3_v2(body,file,parametros,tipo="full",folder="Transcripts/"):
    #Grabar solo texto
    ## Diferenciar en S3 con el tipo 
    parametros["Key"] = folder+tipo+"-"+file
    parametros["Body"] = body
    ## Se pasa los argumentos directo del diccionario
    res_textfile =  s3.put_object(**parametros)
    res_http = res_textfile["ResponseMetadata"]["HTTPStatusCode"]

    if res_http !=200:
        print("Error HTTP" +str(res_http) +" Posible problema de comunicación al escribir datos en S3")
    print("Se guardo correctamente")


def obten_lista_transcripts(bucket,prefijo,tipo="texto"):
    ## Obtener textos solos por default

    # archivos = s3.list_objects(Bucket=bucket) 
    # lista_audios = [x["Key"] for x in archivos["Contents"]]

    objetos = s3.list_objects_v2(Bucket=bucket, Prefix=prefijo )
    lista_transcripts = [x["Key"] for x in objetos["Contents"]]

    ## La funcion previa no funciona correctamente el parametro StartAfter
    #lista_transcripts = [y for y in lista_transcripts if ("Transcripts/" in y and "full" not in y )]
    #lista_transcripts = {"texto":y for y in lista_transcripts if len(y) > len(prefijo)}
    lista_transcripts = [y for y in lista_transcripts if len(y) > len(prefijo)]
    lista_transcripts = [dict(s3url=y) for y in lista_transcripts if tipo in y]

    return lista_transcripts

def obten_textos(lista_dict_textos,buckets):
    for transcript in lista_dict_textos:
        #obj = s3.Bucket(buckets).Object(transcript["s3url"])
        obj = s3res.Object(buckets, transcript["s3url"])
        transcript["texto"] = obj.get()['Body'].read().decode('utf-8')
    return lista_dict_textos

def monitorea_job_entities(jobId):
    #'a3bb945513fcface4fbacfaeb464856f',
    print(jobId)
    param_monitor = {"JobId":jobId}
    while True:
        status = nlp.describe_entities_detection_job(**param_monitor)
        if status['EntitiesDetectionJobProperties']['JobStatus'] in ['COMPLETED', 'FAILED']:
            print(status['EntitiesDetectionJobProperties']['JobStatus'])
            break
        print("Creando transcript de audio")
        time.sleep(15)

    #Retornar la ruta de donde se encuentra las entidades reconocidas
    return status['EntitiesDetectionJobProperties']['OutputDataConfig']["S3Uri"]

def monitorea_job_entities_paralelo(lista_transcripts):
    #'a3bb945513fcface4fbacfaeb464856f',
    print("Procesando "+str(len(lista_transcripts)) + "jobs de deteccion de entidades \n")
    while True:
        bool_check = True ## Bandera la cual con un solo job no terminado se vuelve False
        for transcript in lista_transcripts:
            param_monitor = {"JobId":transcript["JobId_DeteccionEntidades"]}
            status = nlp.describe_entities_detection_job(**param_monitor)
            if status['EntitiesDetectionJobProperties']['JobStatus'] not in ['COMPLETED', 'FAILED']:
                bool_check = False
                break
                #print(status['EntitiesDetectionJobProperties']['JobStatus'])
            else:
                transcript["Entities_S3Uri"] = status['EntitiesDetectionJobProperties']['OutputDataConfig']["S3Uri"]
            print("Creando entidades de Transcript")
        if bool_check == True: ## Si nunca cambio de estado 
            break ## Salir de loop ya terminaron todos los job's
        time.sleep(15)
        
        # #Guardar Uri's de todos
        # for transcript in lista_transcripts:
        #     param_monitor = {"JobId":transcript["JobId_DeteccionEntidades"]}
        #     status = nlp.describe_entities_detection_job(**param_monitor)
        #     transcript["Entities_S3Uri"] = status['EntitiesDetectionJobProperties']['OutputDataConfig']["S3Uri"]

    #Retornar las rutas donde se encuentra las entidades reconocidas
    #return status['EntitiesDetectionJobProperties']['OutputDataConfig']["S3Uri"]
    return lista_transcripts


## Procesar cada transcript obtener (Entidades, sentimientos, etc )
def NLP_transcripts(lista_transcripts,buckets,subfolder="NLP"):
    ##Procesar en paralelo las tareas de NLP
    params_nlp = {
    'DataAccessRoleArn': 'arn:aws:iam::031182997863:role/ComprehendDataAccessRoleVozTransferible',
    'InputDataConfig': { 
        #S3Uri: 's3://voz-salida/transcript_prueba_utf8.txt', 
        'S3Uri': '', 
        'InputFormat': 'ONE_DOC_PER_FILE'
    },
    'LanguageCode': 'es', 
    'OutputDataConfig': { 
        'S3Uri': 's3://voz-salida/NLP/'
    },
    'JobName': 'nlp',
    }

    for transcript in lista_transcripts:
        params_nlp["InputDataConfig"]["S3Uri"] = "s3://"+buckets+"/"+transcript["s3url"]
        horaexacta = datetime.now().strftime("%Y-%m-%d-T-%H-%M-%S")
        jobname = "nlp-"+os.path.splitext(os.path.basename(transcript["s3url"]))[0]+"_"+horaexacta
        #jobname = "nlp--"+horaexacta
        params_nlp["JobName"] = jobname
        
        resultado = nlp.start_entities_detection_job(**params_nlp)
        #resultado = monitorea_job_entities(resultado["JobId"])
        transcript["JobId_DeteccionEntidades"] = resultado["JobId"] 
        ## Resultado sera la URI S3 donde reside las entidades detectadas

        ##Terminar abruptamente la sesion
        #return resultado

    return lista_transcripts






def transcribir_audios_bucket(buckete = "voz-entrada",buckets="voz-salida",lista_audios=[]):
    prefix_s3 = "s3://"
    ##Parametros para job de Transcribe
    params = {'TranscriptionJobName':'string',
    'LanguageCode':'es',
    #MediaSampleRateHertz=123,
    'MediaFormat':'mp3',
    'Media':{
        'MediaFileUri': ''
    },
    'MediaFormat':'mp3',
    'TranscriptionJobName': ''
    }

    ## Parametros para insertar en S3
    params_textfile = {
        'ACL' :'private',
        'Body':b'bytes',
        'Bucket' : 'voz-salida',
        'Key' : 'Transcripts/',
        'ContentLanguage':'es_US',
        'ContentEncoding' : 'UTF-8'
    }

    ## Prefijo de url de S3
    prefix_s3 = "s3://"
    
    lista_dict_audios = []
    dict_audios={}
    
    ### Procesar todos los audios secuencialmente
    for audio in lista_audios:
        dict_audios["MediaFileUri"] = prefix_s3 + buckete + "/" + audio

        params["Media"]["MediaFileUri"] = prefix_s3 + buckete + "/" + audio
        
        #Obtener hora exacta para que no choquen nombres de jobs repetidos
        horaexacta = datetime.now().strftime("%Y-%m-%d-T-%H-%M-%S")
        params["TranscriptionJobName"] =  audio+horaexacta+".txt"
        dict_audios["TranscriptionJobName"] = params["TranscriptionJobName"]

        # Lanzar job de Transcrpcion 
        res = tr.start_transcription_job(**params)

        print(params)
        print("\n")

        #Obtener jobname y revisar status y esperar hasta que finalice
        # Cada 5 segundos se revisa si ya termino (Al terminar se tiene la url del transcript)
        res_url = obten_url_transcript(res["TranscriptionJob"]["TranscriptionJobName"])

        # Realizar consulta de transcript en URL
        full_transcript, text_transcript = consulta_transcript(res_url)

        print(text_transcript)
        print("\n")

        #Guardar en diccionario
        dict_audios["s3Transcript_full"] = full_transcript
        dict_audios["s3Trancript_text"] = text_transcript

        almacenaS3(full_transcript,text_transcript,audio+horaexacta,params_textfile)

        lista_dict_audios.append(dict_audios)
        #tr.start_transcription_job()
        
        ## Prueba solo procesar un audio
        #return lista_dict_audios

    return lista_dict_audios


def transcribir_audios_bucket_paralelo(buckete = "voz-entrada",buckets="voz-salida",lista_audios=[]):
    prefix_s3 = "s3://"

    ##Parametros para job de Transcribe
    params = {'TranscriptionJobName':'string',
    'LanguageCode':'es-US',
    #MediaSampleRateHertz=123,
    'MediaFormat':'mp3',
    'Media':{
        'MediaFileUri': ''
    },
    'MediaFormat':'mp3',
    'TranscriptionJobName': ''
    }

    ## Parametros para insertar en S3
    params_textfile = {
        'ACL' :'private',
        'Body':b'bytes',
        'Bucket' : 'voz-salida',
        'Key' : 'Transcripts/',
        'ContentLanguage':'es_US',
        'ContentEncoding' : 'UTF-8'
    }

    ## Prefijo de url de S3
    prefix_s3 = "s3://"
    lista_dict_audios = []
    dict_audios={}
    
    ### Procesar todos los audios sin esperar terminacion
    for audio in lista_audios:
        dict_audios["MediaFileUri"] = prefix_s3 + buckete + "/" + audio
        params["Media"]["MediaFileUri"] = prefix_s3 + buckete + "/" + audio

        print("Procesando audio"+dict_audios["MediaFileUri"])
        
        #Obtener hora exacta para que no choquen nombres de jobs repetidos
        horaexacta = datetime.now().strftime("%Y-%m-%d-T-%H-%M-%S")
        params["TranscriptionJobName"] =  audio+horaexacta+".txt"
        dict_audios["TranscriptionJobName"] = params["TranscriptionJobName"]

        # Lanzar job de Transcrpcion 
        res = tr.start_transcription_job(**params)
        print(params)
        print("\n")

            
        lista_dict_audios.append(dict_audios.copy())## Anexar una copia modificada de dict_audios
        #Esperar 2 segundos
        time.sleep(2)

    return lista_dict_audios

def revisa_jobs_transcripts(lista_dict):
    ### Revisar que todos acabaron  o tronaron
    print("Esperando finalizacion jobs")
    while True: ## Loop infinito para revision de estatus
        bool_check = True ## Bandera la cual con un solo job no terminado se vuelve False
        
        for item in lista_dict:
            status = tr.get_transcription_job(TranscriptionJobName=item["TranscriptionJobName"])
            if status['TranscriptionJob']['TranscriptionJobStatus'] not in ['COMPLETED', 'FAILED']:
                bool_check = False
                print(item["TranscriptionJobName"] + "No ha terminado")
                break # Ya no checar mas si alguno no ha terminado
            print("Job "+ item["TranscriptionJobName"] + " Termino")
        if bool_check == True: ## Si nunca cambio de estado 
            break ## Salir de loop ya terminaron todos los job's
        print("Espera 45 segundos mas")
        time.sleep(45) ## ## Con que alguno no ha terminado esperar otros 15 segundos para revisar todos

    print("Obteniendo URi's")
    ## Almacearn Urls de todos los transcripts
    for item in lista_dict:
        status = tr.get_transcription_job(TranscriptionJobName=item["TranscriptionJobName"])
        item["url_transcripts"] = status['TranscriptionJob']['Transcript']["TranscriptFileUri"]
    ## Se modifica el diccionario (se les agrega el Uri del transcript en el correspondiente audio)     

    return lista_dict
    # prueba = revisa_jobs_transcripts(lista_dict_audios)


def procesa_transcripts(lista_dict,folder="Transcripts/"):
    params_textfile = {
        'ACL' :'private',
        'Body':b'bytes',
        'Bucket' : 'voz-salida',
        'Key' : 'Transcripts/',
        'ContentLanguage':'es_US',
        'ContentEncoding' : 'UTF-8'
    }

    #Espera a que terminen todos los jobs y obtener las url's de todos
    #lista_dict = revisa_jobs_transcripts(lista_dict)

    for item in lista_dict:
        print(item["url_transcripts"])
        full_transcript, text_transcript = consulta_transcript(item["url_transcripts"])
        item["s3Transcript_full"] = full_transcript
        item["s3Trancript_text"] = text_transcript

        almacenaS3_v2(full_transcript,item["TranscriptionJobName"],params_textfile,tipo="full",folder=folder)
        almacenaS3_v2(text_transcript,item["TranscriptionJobName"],params_textfile,tipo="texto",folder=folder)
    
    return lista_dict
    #prueba = procesa_transcripts(lista_dict_audios)

    ## Obtener todos los resultados de jobs de deteccione de Entidades       
    def obten_lista_nlptrans(bucket,prefijo):
        ## Obtener textos solos por default
        objetos = s3.list_objects_v2(Bucket=bucket, Prefix=prefijo )
        lista_NLP = [x["Key"] for x in objetos["Contents"]]

        #Output de entities siempre es un archivo output.tar    
        lista_NLP = [y for y in lista_NLP if "output.tar" in y]
        lista_NLP = [dict(s3url=y) for y in lista_NLP if prefijo in y]

In [5]:
bucket_entrada = "voz-entrada"
bucket_salida = "voz-salida"
args = dict(procesaaudios=True,paralelo=True,buckete=bucket_entrada,buckets=bucket_salida)

In [ ]:
#Obtener argumentos de consola
args = parser.parse_args()

## Obligatorio
if not (args.buckete):
    sys.exit("Aborta ...No se paso de parametro el bucket '--buckete' donde residen los audios")
bucket_entrada = args.buckete

if (args.buckets):
    bucket_salida = args.buckets
else:
    print("Se usara bucket default para salida: 'voz-salida' \n")
    bucket_salida = 'voz-salida'

In [ ]:
## Obten la lista de objetos de audio 
if(args.procesaaudios):
    print("Procesando Audios.... \n")
    lista_audios = obten_lista_audios(bucket = bucket_entrada)
    print(lista_audios)
    ## Mandar jobs de transcript en paralelo
    if args.paralelo:
        print("Procesa en paralelo")
        lista_dict_audios =transcribir_audios_bucket_paralelo(buckete=bucket_entrada, buckets = bucket_salida,lista_audios=lista_audios )
        ## Espera finalizacion de todos los jobs y obtiene urls de todos los jobs
        lista_dict_audios = revisa_jobs_transcripts(lista_dict_audios)
        lista_dict_audios = procesa_transcripts(lista_dict_audios)

    else:    
        print("Procesa secuencialmente")
        resultado = transcribir_audios_bucket(buckete=bucket_entrada, buckets = bucket_salida,lista_audios=lista_audios )

if(args.procesatranscripts):
    print("Procesando Transcripts.... \n")
    lista_transcripts = obten_lista_transcripts(bucket = bucket_salida,prefijo="Transcripts/")
    lista_transcripts = NLP_transcripts(lista_transcripts,buckets=bucket_salida,subfolder="NLP")
    #lista_transcripts = obten_textos(lista_transcripts,buckets="voz-salida") ## Posiblemente no necesario anexar al diccionario
    lista_transcripts = monitorea_job_entities_paralelo(lista_transcripts)

    print(lista_transcripts)
    #NLP_transcripts()

## Procesar Audios

In [27]:
lista_audios = obten_lista_audios(bucket=bucket_entrada)

In [28]:
lista_audios = lista_audios[0:3]

In [29]:
lista_audios

['33-Punto-Quiebre.mp3', 'ByteNoticias10.mp3', 'ByteNoticias2.mp3']

In [30]:
lista_dict_audios =transcribir_audios_bucket_paralelo(buckete=bucket_entrada, buckets = bucket_salida,lista_audios=lista_audios )

Procesando audios3://voz-entrada/33-Punto-Quiebre.mp3
{'TranscriptionJobName': '33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt', 'LanguageCode': 'es-US', 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 's3://voz-entrada/33-Punto-Quiebre.mp3'}}


Procesando audios3://voz-entrada/ByteNoticias10.mp3
{'TranscriptionJobName': 'ByteNoticias10.mp32019-11-04-T-11-37-23.txt', 'LanguageCode': 'es-US', 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 's3://voz-entrada/ByteNoticias10.mp3'}}


Procesando audios3://voz-entrada/ByteNoticias2.mp3
{'TranscriptionJobName': 'ByteNoticias2.mp32019-11-04-T-11-37-26.txt', 'LanguageCode': 'es-US', 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 's3://voz-entrada/ByteNoticias2.mp3'}}




Vigilar jobs de transcripcion

In [31]:
lista_dict_audios = revisa_jobs_transcripts(lista_dict_audios)

Esperando finalizacion jobs
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txtNo ha terminado
Espera 45 segundos mas
Job 33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt Termino
Job ByteNoticias10.mp32019-11-04-T-11-37-23.txt Termino
Job ByteNoticias2.mp32019-11-04-T-11-37-26.txt Termino
Obteniendo URi's


Guardar transcripts en S3

In [32]:
lista_dict_audios = procesa_transcripts(lista_dict_audios)

https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/031182997863/33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt/c179faf7-bf18-4bf8-b9c1-9c4d3283570c/asrOutput.json?X-Amz-Security-Token=AgoJb3JpZ2luX2VjEDEaCXVzLXdlc3QtMiJGMEQCIGkgTSI8K7AwhRlS7x0i8%2F6276PmDbs4AqQwI%2F8%2F8Zz1AiBcAb0hWQlY6Q8ktzdXBJcnqndKSk7hagTXGgYZb03JWCr8AwhKEAEaDDA4MDI0ODMyMjIwNiIMinaZp7mPFyd%2BZ%2B0oKtkDtqcwKpRTc51%2BBIwKV2pFmJ5jmds46hh7QQ17uvXieUkFx%2FIEW%2BQl19m8M1NaL%2BmKFUYNHIFqRqH93Bx3gfuYG%2FRrX43iw%2Fk9re5f15un%2BediB7%2FriffvY2lXG0nx1BLtwvMMaxmupIywGmzmA%2Bnkhmiq1cbEXkyrmMJA2PGyODYAExNzWHjAnwxtfcHU2RFDvwazNRwFzpwIt%2BfTFQSQgzXpr%2BmNlnPpzsnbxlPcKeB4Et%2BbrNhbnTnp87EX8NpxX2e%2FmHSUEpr4NnJ%2FE5hQUzYMVgguvi6cSamielY%2Bs26dN4mXTdhLnKXGE%2FZXrURJOhLzVr%2BEdORb9nXtY53NO5LQTtAj1Q2ebiewI6SBm2UGdY3mTw1wWsDmrzhTPmLDxZRV%2BoQJmlMHlrRb9q9vUHt9%2BHNigo0j8reimNnYedPTUL3OAuehjkLWeif%2Bt0E%2Bor0xtJX5OHSZ1DI5VRAMRkPQqkrNAs7TcxEDmEmWG9mqKv%2F3NRJXy%2BD1Qy6GPdBLvue0zWj3D2gbfRQadio0Hw3c3Fk6yzxxbK2KLJQkcpgsYHBhWOoAyPa

C:\Users\adrian.rodriguez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Se guardo correctamente
Se guardo correctamente
https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/031182997863/ByteNoticias10.mp32019-11-04-T-11-37-23.txt/a658c94e-0aab-4cc3-ab66-1b25d4ae12ec/asrOutput.json?X-Amz-Security-Token=AgoJb3JpZ2luX2VjEDAaCXVzLXdlc3QtMiJIMEYCIQC2u0j4z10srXx5KFPgZlqVb9cNbauU5kw4PAEFu8MBLQIhALqQSJZhg9%2BpUTvMN8Czh%2BPp%2FwT%2FVNav2j6%2BIeZDscXHKvwDCEkQARoMMDgwMjQ4MzIyMjA2IgxRP%2Fb6yKHTPbSTb1gq2QP5gDY%2B2nGAdl%2FUwjZX1MuAYTJb0YH4ZjQKkzsgCuLyd3h6ZOoL67kyHRvyTVDgl2l%2B267zx%2FgMiYdRWoholMXdZlerilapZaRN6mSgD%2BZgGXnbrZWHZJ9kjIMXV5TzfLu76TaozTTevdI8QL53DNJgw11tEb3wZkmz0uxlkpYOlyK5W6U1%2FJMECukGv%2BDIymEmk3y%2FbgdnKLtULVAZsD0Bi2TO0dn5E3n6BZGASlLnNHzc9UodsU%2Bha7UTEZfYXiNtHQceFfKMo7Lp5p3aYMeCRDn9nuvXKIcy1uoeA3xTkucZsId2ncJ%2Bu24EC9NAtHhwKM9a8pn2T93Yhj0UT0B1T9On5qxSICiaq0QDh4mpn1lct0crvc7r3a7uTM%2BxlfTGvRHjFRXIkJXk05O3%2FphhvsDyA5o3zxXqi71EdL%2BsE%2FVnuAUQe5O5h5kpo9n2RsZxcUdRHUgZ0SXi%2B66UdZyyYo9kYSQRpNgu6KKU724foYHE%2FMt0dfMTF1e7MD631z7AWV8w3RTWXR7HP7B5o

C:\Users\adrian.rodriguez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Se guardo correctamente
Se guardo correctamente
https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/031182997863/ByteNoticias2.mp32019-11-04-T-11-37-26.txt/1cd513a2-785b-4e59-98e0-3291c0263cad/asrOutput.json?X-Amz-Security-Token=AgoJb3JpZ2luX2VjEDAaCXVzLXdlc3QtMiJIMEYCIQDVg1uMXoSFpeKy2inxXwqNLCp2RpN6ZKAcvrKNALz3bQIhANkDM3HHpOXCk%2FKf3Z%2FZQWmwfzdTgw9AcKePhgAffiYAKvwDCEkQARoMMDgwMjQ4MzIyMjA2IgwelS4wmV%2F1ieVT2FUq2QPUFEfj8nCxMAqAPh8m0KCUct2F9P1IQxvXNgEVx%2BQ1CerFd2pdxFTJ96q2MR%2F4h9ffFO7L4LF2gofDITlxKowFfQZYY8BgXAR2ZvNkucYPWFP2CLyqxDPQWTBiE0D1H3qyGDSs%2BqQ79r11m0dKXHo5uNn0IJIb2IIFSkgWiLGKwBpIg0UZ2LJ5jX8YLgoz0dpyoND5Bsb9nnD1PDdUaqJxsYqEWqd0Sj4rzqRsVlo0rYeltY3IkSpKv9y1b11F9vQXtOxJBOSRMYo069mQS071ywgCKCLFquXoIrraVZpqMUmdxtqGV%2BOyQ1fY8d68AynOaDTo%2BWSe16vwyj6NtTWQQGHaMtkgfHFdL1kmu9j2%2Bku95nYRxzFqctsSX92%2BXVR1itdbW5Ptk%2FPBXU8K5o6Kx5CD9vPtkqnuTkWhn3uhE%2Fj24PI%2Bi46%2Fv5BmA0kz6%2B41zxde90rRaz3P%2FRZTHzruCnFC81tSW2SyFc2s4WUQg44dOYnl6iHqmOyvxMliopLTLWu3W81m%2Buocm76tQ6y75t6cJSqR

C:\Users\adrian.rodriguez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Se guardo correctamente
Se guardo correctamente


In [33]:
lista_dict_audios

[{'MediaFileUri': 's3://voz-entrada/33-Punto-Quiebre.mp3',
  'TranscriptionJobName': '33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt',
  'url_transcripts': 'https://s3.us-west-2.amazonaws.com/aws-transcribe-us-west-2-prod/031182997863/33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt/c179faf7-bf18-4bf8-b9c1-9c4d3283570c/asrOutput.json?X-Amz-Security-Token=AgoJb3JpZ2luX2VjEDEaCXVzLXdlc3QtMiJGMEQCIGkgTSI8K7AwhRlS7x0i8%2F6276PmDbs4AqQwI%2F8%2F8Zz1AiBcAb0hWQlY6Q8ktzdXBJcnqndKSk7hagTXGgYZb03JWCr8AwhKEAEaDDA4MDI0ODMyMjIwNiIMinaZp7mPFyd%2BZ%2B0oKtkDtqcwKpRTc51%2BBIwKV2pFmJ5jmds46hh7QQ17uvXieUkFx%2FIEW%2BQl19m8M1NaL%2BmKFUYNHIFqRqH93Bx3gfuYG%2FRrX43iw%2Fk9re5f15un%2BediB7%2FriffvY2lXG0nx1BLtwvMMaxmupIywGmzmA%2Bnkhmiq1cbEXkyrmMJA2PGyODYAExNzWHjAnwxtfcHU2RFDvwazNRwFzpwIt%2BfTFQSQgzXpr%2BmNlnPpzsnbxlPcKeB4Et%2BbrNhbnTnp87EX8NpxX2e%2FmHSUEpr4NnJ%2FE5hQUzYMVgguvi6cSamielY%2Bs26dN4mXTdhLnKXGE%2FZXrURJOhLzVr%2BEdORb9nXtY53NO5LQTtAj1Q2ebiewI6SBm2UGdY3mTw1wWsDmrzhTPmLDxZRV%2BoQJmlMHlrRb9q9vUHt9%2BHNigo0j8re

## Procesar Transcripts

In [36]:
bucket_salida

'voz-salida'

In [38]:
lista_transcripts = obten_lista_transcripts(bucket = bucket_salida,prefijo="Transcripts/")

In [39]:
lista_transcripts

[{'s3url': 'Transcripts/texto-33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt'},
 {'s3url': 'Transcripts/texto-ByteNoticias10.mp32019-11-04-T-11-37-23.txt'},
 {'s3url': 'Transcripts/texto-ByteNoticias2.mp32019-11-04-T-11-37-26.txt'}]

In [46]:
lista_transcripts = NLP_transcripts(lista_transcripts,buckets=bucket_salida,subfolder="NLP")

In [49]:
lista_transcripts

[{'s3url': 'Transcripts/texto-33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt',
  'JobId_DeteccionEntidades': '3ccdb49d1ec885755555b02766b9f72a'},
 {'s3url': 'Transcripts/texto-ByteNoticias10.mp32019-11-04-T-11-37-23.txt',
  'JobId_DeteccionEntidades': '309206cb1edb2984dec3fb13446efdac'},
 {'s3url': 'Transcripts/texto-ByteNoticias2.mp32019-11-04-T-11-37-26.txt',
  'JobId_DeteccionEntidades': '2cdf641af0d036b278d2db4bc752e1bf'}]

In [51]:
lista_transcripts = monitorea_job_entities_paralelo(lista_transcripts)

Procesando 3jobs de deteccion de entidades 

Creando entidades de Transcript
Creando entidades de Transcript
Creando entidades de Transcript
Creando entidades de Transcript
Creando entidades de Transcript


In [53]:
lista_transcripts

[{'s3url': 'Transcripts/texto-33-Punto-Quiebre.mp32019-11-04-T-11-37-21.txt',
  'JobId_DeteccionEntidades': '3ccdb49d1ec885755555b02766b9f72a',
  'Entities_S3Uri': 's3://voz-salida/NLP/031182997863-NER-3ccdb49d1ec885755555b02766b9f72a/output/output.tar.gz'},
 {'s3url': 'Transcripts/texto-ByteNoticias10.mp32019-11-04-T-11-37-23.txt',
  'JobId_DeteccionEntidades': '309206cb1edb2984dec3fb13446efdac',
  'Entities_S3Uri': 's3://voz-salida/NLP/031182997863-NER-309206cb1edb2984dec3fb13446efdac/output/output.tar.gz'},
 {'s3url': 'Transcripts/texto-ByteNoticias2.mp32019-11-04-T-11-37-26.txt',
  'JobId_DeteccionEntidades': '2cdf641af0d036b278d2db4bc752e1bf',
  'Entities_S3Uri': 's3://voz-salida/NLP/031182997863-NER-2cdf641af0d036b278d2db4bc752e1bf/output/output.tar.gz'}]